In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.discriminant_analysis import StandardScaler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

In [116]:
random_state=123

In [117]:
# Read in the data
df = pd.read_csv('../Dataset/cleaned_hypertension_data.csv')
df

,Class,Level_of_Hemoglobin,Age,BMI,Sex,Smoking,Physical_activity,salt_content_in_the_diet,Level_of_Stress,Chronic_kidney_disease,Adrenal_and_thyroid_disorders,Genetic_Pedigree_Coefficient,alcohol_consumption_per_day
0,1,11.28,34,23,1,0,45961,48071,2,1,1,0.90,336.333333
1,0,9.75,54,33,1,0,26106,25333,3,0,0,0.23,205.000000
2,1,10.79,70,49,0,0,9995,29465,2,1,0,0.91,67.000000
3,0,11.00,71,50,0,0,10635,7439,1,1,0,0.43,242.000000
4,1,14.17,52,19,0,0,15619,49644,2,0,0,0.83,397.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,10.14,69,26,1,1,26118,47568,3,1,0,0.02,144.000000
1996,1,11.77,24,45,1,1,2572,8063,3,1,1,1.00,299.666667
1997,1,16.91,18,42,0,0,14933,24753,2,1,1,0.22,369.000000
1998,0,11.15,46,45,1,1,18157,15275,3,0,1,0.72,253.000000


In [118]:
from imblearn.under_sampling import ClusterCentroids, TomekLinks
from imblearn.over_sampling import RandomOverSampler

def splitting_data(df, sampling):
    X = df.drop(['Class'], axis=1)
    y = df['Class']

    if sampling == 'none':
        return X, y
    elif sampling == 'SMOTEENN':
        sampler = SMOTEENN(random_state=0)
    elif sampling == 'SMOTE':
        sampler = SMOTE(random_state=0)
    elif sampling == 'under':
        sampler = RandomUnderSampler(random_state=0)
    elif sampling == 'over':
        sampler = RandomOverSampler(random_state=0)
    elif sampling == 'cluster_centroids':
        sampler = ClusterCentroids(random_state=0)
    elif sampling == 'tomek_links':
        sampler = TomekLinks()

    X_resampled, y_resampled = sampler.fit_resample(X, y)
    return X_resampled, y_resampled


In [119]:
def training(X_train, y_train):
    # Create a KNN classifier with 5 neighbors
    DT = DecisionTreeClassifier(random_state=random_state)
    # Fit the classifier to the data
    DT.fit(X_train, y_train)
    return DT

In [120]:
def predict(DT, X_test ,y_test):
    # Predict the labels for the training data X
    y_pred = DT.predict(X_test)
    cr=classification_report(y_test, y_pred)
    print(cr)

In [121]:
# def optimize(X,y):
#     k_values = [i for i in range (1,31)]
#     scores = []

#     scaler = StandardScaler()
#     X = scaler.fit_transform(X)

#     for k in k_values:
#         knn = KNeighborsClassifier(n_neighbors=k)
#         score = cross_val_score(knn, X, y, cv=5)
#         scores.append(np.mean(score))

#     best_index = np.argmax(scores)
#     best_k = k_values[best_index]

#     return best_k, scores
        

In [122]:




from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def optimize_with_grid(X_train, y_train):
    # Define a pipeline that first scales the data and then applies the classifier
    pipe = Pipeline([
        # ('scaler', StandardScaler()),
        ('dt', DecisionTreeClassifier(random_state=random_state))
    ])

    # Define the parameter grid to search
    param_grid = {
        'dt__max_depth': [None, 10, 20, 30, 40, 50],
        'dt__min_samples_split': [2, 5, 10],
        'dt__min_samples_leaf': [1, 2, 4],
        'dt__criterion': ['gini', 'entropy']
    }

    # Create the GridSearchCV object
    DT_cv = GridSearchCV(pipe,param_grid, cv=5, verbose=1, n_jobs=-1)

    # Perform the grid search on the provided data
    DT_cv.fit(X_train, y_train)

    # Best parameters and best score
    best_params = DT_cv.best_params_
    best_score = DT_cv.best_score_
    best_estimator = DT_cv.best_estimator_
    print(best_params)
    print(best_score)

    return best_estimator


<h1> DT on original data with optimization </h1>

In [123]:
# using function with no sampling 
X, y= splitting_data(df, 'none')
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [124]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y.value_counts())

Number of observations in each class in the training set:
0    1013
1     987
Name: Class, dtype: int64


In [125]:
DT1 =training(X_train, y_train)
y_pred = predict(DT1, X_test, y_test)

              precision    recall  f1-score   support

           0       0.81      0.85      0.83       191
           1       0.86      0.82      0.84       209

    accuracy                           0.84       400
   macro avg       0.84      0.84      0.84       400
weighted avg       0.84      0.84      0.84       400



In [126]:
best_DT1 = optimize_with_grid(X_train, y_train)
prediction = predict(best_DT1, X_test, y_test)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
{'dt__criterion': 'entropy', 'dt__max_depth': 10, 'dt__min_samples_leaf': 4, 'dt__min_samples_split': 10}
0.85125
              precision    recall  f1-score   support

           0       0.84      0.87      0.85       191
           1       0.88      0.85      0.86       209

    accuracy                           0.86       400
   macro avg       0.86      0.86      0.86       400
weighted avg       0.86      0.86      0.86       400



<h1> DT using SMOTE sampling </h1>

In [127]:
X,y = splitting_data(df, 'SMOTE')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [128]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y.value_counts())

Number of observations in each class in the training set:
1    1013
0    1013
Name: Class, dtype: int64


In [129]:
DT2 =training(X_train, y_train)
y_pred = predict(DT2, X_test, y_test)

              precision    recall  f1-score   support

           0       0.83      0.82      0.83       211
           1       0.81      0.82      0.81       195

    accuracy                           0.82       406
   macro avg       0.82      0.82      0.82       406
weighted avg       0.82      0.82      0.82       406



In [130]:
best_DT2 = optimize_with_grid(X_train, y_train)
prediction = predict(best_DT2, X_test, y_test)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
{'dt__criterion': 'gini', 'dt__max_depth': 10, 'dt__min_samples_leaf': 1, 'dt__min_samples_split': 5}
0.8456790123456791
              precision    recall  f1-score   support

           0       0.84      0.87      0.86       211
           1       0.86      0.83      0.84       195

    accuracy                           0.85       406
   macro avg       0.85      0.85      0.85       406
weighted avg       0.85      0.85      0.85       406



<h1> DT using SMOTEENN sampling </h1>

In [131]:
X,y = splitting_data(df, 'SMOTEENN')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [132]:
print("Number of observations in each class in the set:")
print(y.value_counts())

Number of observations in each class in the set:
1    151
0    137
Name: Class, dtype: int64


In [133]:
knn5 =training(X_train, y_train)
y_pred = predict(knn5, X_test, y_test)

              precision    recall  f1-score   support

           0       0.68      0.76      0.72        25
           1       0.80      0.73      0.76        33

    accuracy                           0.74        58
   macro avg       0.74      0.74      0.74        58
weighted avg       0.75      0.74      0.74        58



In [134]:
best_knn = optimize_with_grid(X_train, y_train)
prediction = predict(best_knn, X_test, y_test)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
{'dt__criterion': 'gini', 'dt__max_depth': None, 'dt__min_samples_leaf': 1, 'dt__min_samples_split': 5}
0.8304347826086957
              precision    recall  f1-score   support

           0       0.67      0.72      0.69        25
           1       0.77      0.73      0.75        33

    accuracy                           0.72        58
   macro avg       0.72      0.72      0.72        58
weighted avg       0.73      0.72      0.73        58



<h1> DT on Random undersampling </h1>

In [135]:
X,y = splitting_data(df, 'under')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [136]:
print("Number of observations in each class in the set:")
print(y.value_counts())

Number of observations in each class in the set:
0    987
1    987
Name: Class, dtype: int64


In [137]:
knn7 =training(X_train, y_train)
y_pred = predict(knn7, X_test, y_test)

              precision    recall  f1-score   support

           0       0.80      0.86      0.83       200
           1       0.84      0.78      0.81       195

    accuracy                           0.82       395
   macro avg       0.82      0.82      0.82       395
weighted avg       0.82      0.82      0.82       395



In [138]:
best_knn = optimize_with_grid(X_train, y_train)
prediction = predict(best_knn, X_test, y_test)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
{'dt__criterion': 'gini', 'dt__max_depth': 10, 'dt__min_samples_leaf': 1, 'dt__min_samples_split': 2}
0.8492726542093629
              precision    recall  f1-score   support

           0       0.79      0.87      0.83       200
           1       0.85      0.76      0.80       195

    accuracy                           0.82       395
   macro avg       0.82      0.81      0.81       395
weighted avg       0.82      0.82      0.81       395



<h1> DT on Cluster Centroids </h1>

In [139]:
X,y = splitting_data(df, 'cluster_centroids')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [140]:
print("Number of observations in each class in the set:")
print(y.value_counts())

Number of observations in each class in the set:
0    987
1    987
Name: Class, dtype: int64


In [141]:
knn7 =training(X_train, y_train)
y_pred = predict(knn7, X_test, y_test)

              precision    recall  f1-score   support

           0       0.83      0.86      0.84       200
           1       0.85      0.82      0.83       195

    accuracy                           0.84       395
   macro avg       0.84      0.84      0.84       395
weighted avg       0.84      0.84      0.84       395



In [142]:
best_knn = optimize_with_grid(X_train, y_train)
prediction = predict(best_knn, X_test, y_test)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
{'dt__criterion': 'entropy', 'dt__max_depth': 10, 'dt__min_samples_leaf': 1, 'dt__min_samples_split': 2}
0.8600321478802491
              precision    recall  f1-score   support

           0       0.80      0.87      0.83       200
           1       0.85      0.77      0.81       195

    accuracy                           0.82       395
   macro avg       0.83      0.82      0.82       395
weighted avg       0.83      0.82      0.82       395



<h1> DT on Tomek Links </h1>

In [143]:
X,y = splitting_data(df, 'tomek_links')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [144]:
print("Number of observations in each class in the set:")
print(y.value_counts())

Number of observations in each class in the set:
1    987
0    694
Name: Class, dtype: int64


In [145]:
knn7 =training(X_train, y_train)
y_pred = predict(knn7, X_test, y_test)

              precision    recall  f1-score   support

           0       0.80      0.81      0.80       139
           1       0.86      0.86      0.86       198

    accuracy                           0.84       337
   macro avg       0.83      0.83      0.83       337
weighted avg       0.84      0.84      0.84       337



In [146]:
best_knn = optimize_with_grid(X_train, y_train)
prediction = predict(best_knn, X_test, y_test)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
{'dt__criterion': 'entropy', 'dt__max_depth': 10, 'dt__min_samples_leaf': 1, 'dt__min_samples_split': 10}
0.8512067913221995
              precision    recall  f1-score   support

           0       0.80      0.87      0.83       139
           1       0.90      0.85      0.88       198

    accuracy                           0.86       337
   macro avg       0.85      0.86      0.85       337
weighted avg       0.86      0.86      0.86       337

